# Sequence Alignement and Detecting Motifs

This work was done for the ULB course: Computational biology and bioinformatics (INFO-F-439) by Thomas Van Gysegem (all rigth reserved).

First of all, let us introduce some utility function that will be used for this project. The first one is a function that will be used to display our Dynamic Programing matrice:

In [131]:
%%html
// Some style to get a better display of our data
<style>
.outter td, .outter, .outter tr, .outter th {padding: 1px;}
.inner td, .inner, .inner tr, .inner th {border:none!important;}
.inner td, .inner tr, .inner th {width: 20px; height: 20px}
.optimal td, .optimal, .optimal tr, .optimal th {border:2px!important; color: red;}
</style>

In [27]:
from IPython.display import HTML, display

def display_dp_matrix(dp_matrix, x, y, backtrack_matrix, optimal_path):
    """ Simple display function that shows all informations about the DP matrice, sequences, ... """
    
    html_str = '<table class="outter">'
    
    # First row (header)
    html_str += '<tr><td></td><td></td><td>{}</td></tr>'.format('</td><td>'.join(_ for _ in y))
    
    for i in range(len(dp_matrix)):
        row = dp_matrix[i]
        
        c = '' if i <= 0 else x[i - 1]
        
        html_str += '<tr><td>{}</td>'.format(c)
        
        for j in range(len(row)):
            value = row[j]
            
            upleft = '<img src="upleft.png" height="42" width="9" />' if backtrack_matrix[i][j][0] else ''
            up = '<img src="up.png" height="9" width="9" />' if backtrack_matrix[i][j][1] else ''
            left = '<img src="left.png" height="9" width="9" />' if backtrack_matrix[i][j][2] else ''
            
            upleft = '<img src="upleftred.png" height="42" width="9" />' if optimal_path_direction[i][j][0] else upleft
            up = '<img src="upred.png" height="9" width="9" />' if optimal_path_direction[i][j][1] else up
            left = '<img src="leftred.png" height="9" width="9" />' if optimal_path_direction[i][j][2] else left
            
            optimal = ' optimal' if optimal_path[i][j] else ''
            
            sub_table = '<table class="inner{}"><tr><td>{}</td><td>{}</td></tr>'.format(optimal, upleft, up)
            sub_table += '<tr><td>{}</td><td>{}</td></tr></table>'.format(left, int(value))
            
            html_str += '<td>{}</td>'.format(sub_table)
        
        html_str += '</tr>'
    
    html_str += '</table>'
    
    display(HTML(html_str))

The second one will be used to parse BLOSUM and PAM matrices used. For simplicity sake we will use a 2D Python dictionnary:

In [19]:
def parse_ranking_matrix(filename):
    """ Parse a ranking matrix file with comment line beginning with # """
    ranking_matrix = {}
    
    header = []
    metadata_parsed = False
    
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Ignore empty or comment line
            if line == "" or line.startswith('#'):
                continue
            
            # First line of data: Header
            elif not metadata_parsed:
                header = [_ for _ in line if _ != ' ']
                metadata_parsed = True
            
            # Other lines
            else:
                data = line.split(' ')
                
                key1 = data[0]
                data = [_ for _ in data[1:] if _ != '']
                
                ranking_matrix[key1] = {}
                for i in range(len(data)):
                    key2 = header[i]
                    value = int(data[i])
                    
                    ranking_matrix[key1][key2] = value
        
        return ranking_matrix

This function is used to display solutions with human readable formatting (showing gap, matches and so on)

In [42]:
def display_solution(solution):
    lines = ['', '', '']
    
    pi, pj = 0, 0
    for i, j in solution[::-1][1:]:
        # Matched
        if (i - pi) + (j - pj) == 2:
            lines[0] += '<td><b>{}</b></td>'.format(x[i-1])
            lines[1] += '<td>|</td>'
            lines[2] += '<td><b>{}</b></td>'.format(y[j-1])
        elif (i - pi) == 1:
            lines[0] += '<td>{}</td>'.format(x[i-1])
            lines[1] += '<td></td>'
            lines[2] += '<td>-</td>'
        else:
            lines[0] += '<td>-</td>'
            lines[1] += '<td></td>'
            lines[2] += '<td>{}</td>'.format(y[j-1])
        
        pi, pj = i, j
    
    lines[0] = '<table class="inner"><tr>{}</tr>'.format(lines[0])
    lines[1] = '<tr>{}</tr>'.format(lines[1])
    lines[2] = '<tr>{}</tr></table>'.format(lines[2])
    
    display(HTML(''.join(lines)))

The variables used are as follows:
* `dp_matrix` is the main matrix computed with a recursive definition of the value of each cells like seen in the course.
* `q` is the ranking matrix (BLOSOM, PAM, ... depending which one you choosed to parse
* gap_penalty is, of course, the penalty for introducing a gap
* bactrack_matrix is a somewhat hacky matrix that stores which way we come from for a given cell of the dp_matrix. Each cell is a list of 3 boolean, the first one is True if we come from the upper left cell (so a match occured), the second one is True if we introduced a gap in Y and the last one is True if we introduced a gap in X.
* optimal_path is a list of boolean, True if one optimal solution goes through it, False otherwise. It is used purely for display
* optimal_path_direction is like backtrack_matrix but for showing red arrows instead of black ones.
* l_solutions is a list of list of ordered positions in dp_matrix, each list in l_solutions is an optimal solution (up to k solutions)

## Part 1: Sequence Alignement Algorithms

This is the first part of the assignement.

### Needleman-Wunsch (global)

First, please input your sequences here and how many sub-optimal solutions you want (k):

In [21]:
x = 'THISLINE'
y = 'ISALIGNED'
k = 3

Then, we need to load a ranking matrix and sepcify a gap penalty value

In [22]:
filename = 'BLOSUM62'
q = parse_ranking_matrix(filename)
gap_penalty = -4

In [23]:
import numpy as np

M = len(x) + 1
N = len(y) + 1

dp_matrix = np.zeros((M, N))

backtrack_matrix = []
optimal_path = []
optimal_path_direction = []
for i in range(M):
    backtrack_matrix.append([])
    optimal_path.append([])
    optimal_path_direction.append([])
    
    for j in range(N):
        backtrack_matrix[i].append([False, False, False])
        optimal_path[i].append(False)
        optimal_path_direction[i].append([False, False, False])

In [24]:
# Initialise DP matrix with gap penalty
for i in range(M):
    dp_matrix[i][0] = gap_penalty * i
    backtrack_matrix[i][0] = [False, True, False]
    
for j in range(N):
    dp_matrix[0][j] = gap_penalty * j
    backtrack_matrix[0][j] = [False, False, True]
    
backtrack_matrix[0][0] = [False, False, False]

In [25]:
# Nested loop to process all the DP matrix
for i in range(1, M):
    for j in range(1, N):
        key1 = x[i - 1]
        key2 = y[j - 1]
        
        no_gap = dp_matrix[i - 1][j - 1] + q[key1][key2]
        gap_left = dp_matrix[i][j - 1] + gap_penalty
        gap_up = dp_matrix[i - 1][j] + gap_penalty
        
        value = max(no_gap, gap_left, gap_up)
        
        dp_matrix[i][j] = value
        
        backtrack_matrix[i][j] = [
            no_gap == value,
            gap_up == value,
            gap_left == value
        ]

In [43]:
# A bit of backtracking now
def backtrack(solution, i, j, ignore):
    solution.append((i, j))
    
    optimal_path[i][j] = True
    
    if i == 0 and j == 0:
        return True
    
    if backtrack_matrix[i][j][0]: 
        if ignore == 0:
            optimal_path_direction[i][j][0] = True
            return backtrack(solution, i-1, j-1, ignore)
        else:
            ignore -=1
    if backtrack_matrix[i][j][1]: 
        if ignore == 0:
            optimal_path_direction[i][j][1] = True
            return backtrack(solution, i-1, j, ignore)
        else:
            ignore -=1
    if backtrack_matrix[i][j][2]: 
        if ignore == 0:
            optimal_path_direction[i][j][2] = True
            return backtrack(solution, i, j-1, ignore)
        else:
            ignore -=1
    
    return False

l_solution = []
for ignore in range(k):
    solution = []
    backtrack(solution, M - 1, N - 1, ignore)
    
    if solution[-1] == (0, 0):
        l_solution.append(solution)

display_dp_matrix(dp_matrix, x, y, backtrack_matrix, optimal_path)

for i in range(len(l_solution)):
    print("Solution #{}:".format(i+1))
    display_solution(l_solution[i])

Solution #1:


T,H,I,S,-,L,I,-,N,E,-
,,|,|,,|,|,,|,|,
-,-,I,S,A,L,I,G,N,E,D


### Smith-Waterman(local)